In [1]:
import tensorflow as tf
from collections import namedtuple
import json
import numpy as np
from read_data import read_data
import os


In [2]:
# d='D:/Wisdom/git/tmp/BIDAF/bi-att-flow-tf1.8/data/squad/data_dev.json'
# with open(d,'r') as json_file:
#     data = json.load(json_file)
# d='D:/Wisdom/git/tmp/BIDAF/bi-att-flow-tf1.8/data/squad/shared_dev.json'
# with open(d,'r') as json_file:
#     shared = json.load(json_file)
config=namedtuple('config',[])
config.data_dir='D:/Wisdom/git/tmp/BIDAF/bi-att-flow-tf1.8/data/squad/'
config.out_dir=config.data_dir+'/out/00/'
config.shared_path='D:/Wisdom/git/tmp/BIDAF/bi-att-flow-tf1.8/data/squad/shared_test.json'
config.lower_word=True
config.finetune=False
config.known_if_glove=True
config.use_glove_for_unk=True
config.word_count_th=10
config.char_count_th=50

In [3]:
N, M, JX, JQ, VW, VC, d, W,dc, dw, dco ='batch_size','max_num_sents','max_sent_size','max_ques_size',\
'word_vocab_size','char_vocab_size','hidden_size','max_word_size','char_emb_size','word_emb_size','char_out_size'

In [4]:
config.char_count_th=50
dataset=read_data(config=config,data_type='dev',ref=False)

Loaded 10570/10570 examples from dev


In [5]:
dco

'char_out_size'

# Reset point

In [6]:
tf.reset_default_graph()
batch_size=5

max_word_size=16
word_emb_size=100

word_vocab_size=1223

x = tf.placeholder('int32', [batch_size, None], name='x')
cx = tf.placeholder('int32', [batch_size, None, max_word_size], name='cx')
x_mask = tf.placeholder('bool', [batch_size, None], name='x_mask')
q = tf.placeholder('int32', [batch_size, None], name='q')
cq = tf.placeholder('int32', [batch_size, None, max_word_size], name='cq')
q_mask = tf.placeholder('bool', [batch_size, None], name='q_mask')
y = tf.placeholder('bool', [batch_size, None], name='y')
y2 = tf.placeholder('bool', [batch_size, None], name='y2')
is_train = tf.placeholder('bool', [], name='is_train')
new_emb_mat = tf.placeholder('float', [None, word_emb_size], name='new_emb_mat')

In [7]:
char_vocab_size=len(dataset.shared['char2idx'])
char_emb_size=8
char_emb_mat=tf.get_variable("char_emb_mat",shape=[char_vocab_size, char_emb_size],dtype='float')

In [8]:
char_emb_mat.shape

TensorShape([Dimension(107), Dimension(8)])

In [9]:
from tensorflow.nn import embedding_lookup

In [10]:
# N = batch_size
# M = max_num_sents
max_num_sents=1
# JX = max_sent_size
max_sent_size=618
#  W = max_word_size
max_word_size=26
# dc = char_emb_size
# JQ = max_ques_size
max_ques_size= 33
Acx = tf.nn.embedding_lookup(char_emb_mat,cx)  # [N, M, JX, W, dc]
Acq = tf.nn.embedding_lookup(char_emb_mat,cq)  # [N, JQ, W, dc]
# Acx = tf.reshape(Acx, [-1, JX, W, dc])
# Acq = tf.reshape(Acq, [-1, JQ, W, dc])
Acx = tf.reshape(Acx, [-1, max_sent_size, max_word_size, char_emb_size])
Acq = tf.reshape(Acq, [-1, max_ques_size, max_word_size, char_emb_size])


In [11]:
print(Acx.shape)
print(Acq.shape)

(?, 618, 26, 8)
(?, 33, 26, 8)


In [12]:
Acx.get_shape()[-1]

Dimension(8)

In [12]:
from tensorflow.nn import conv1d, conv2d

In [13]:
####### charCNN #######
n_channel=8 #Acx.get_shape()[-1] # char_emb_size
filter_size=100
height=5
char_out_size=100
with tf.variable_scope("char"):
    filter_ = tf.get_variable('filter',shape=[1, height, n_channel, filter_size], dtype='float')
    bias_ = tf.get_variable('bias',shape=[filter_size], dtype='float')
    strides = [1, 1, 1, 1]
    padding='VALID'
    xxc = conv2d(Acx, filter_, strides, padding) + bias_  # [N*M, JX, W/filter_stride, d]
    xx = tf.reduce_max(tf.nn.relu(xxc), 2)  # [-1, JX, d]
    tf.get_variable_scope().reuse_variables()
    qqc = conv2d(Acq, filter_, strides, padding) + bias_
    qq = tf.reduce_max(tf.nn.relu(qqc), 2)  
    xx = tf.reshape(xx, [-1, max_num_sents, max_sent_size, char_out_size])
    qq = tf.reshape(qq, [-1, max_ques_size, char_out_size])

In [14]:
####### word2vec #######
word_emb_mat = tf.get_variable("word_emb_mat", shape=[word_vocab_size, word_emb_size], dtype='float')


In [15]:
new_emb_mat.shape

TensorShape([Dimension(None), Dimension(100)])

In [16]:
word_emb_mat = tf.concat([word_emb_mat, new_emb_mat],0)
Ax = embedding_lookup(word_emb_mat, x) 
Aq = embedding_lookup(word_emb_mat, q)

In [17]:
print(Ax.shape)
print(xx.shape)
print(Aq.shape)
print(qq.shape)
print(x.shape)
print(word_emb_mat.shape)

(5, ?, 100)
(?, 1, 400, 100)
(5, ?, 100)
(?, 30, 100)
(5, ?)
(?, 100)


In [21]:
# concatenation of word_vector and char_vector
xx = tf.concat([xx, Ax],3)
qq = tf.concat([qq, Aq],2)

ValueError: Shape must be rank 4 but is rank 3 for 'concat_3' (op: 'ConcatV2') with input shapes: [?,1,400,100], [5,?,100], [].

In [19]:
print(qq.get_shape()[-1])
print(xx.shape)
print(qq.shape)

100
(?, 1, 400, 100)
(?, 30, 100)


In [105]:
xxxx=tf.reshape(xx,[-1, 200])
qqqq=tf.reshape(qq,[-1,200])

In [106]:
print(xxxx.shape)
print(qqqq.shape)

(2000, 200)
(150, 200)


In [107]:
# ####### highway network #######
# highway_num_layers=1
# #     W_T =tf.get_variable("weight_transform", shape=[d, d], dtype='float')
# #     b_T = tf.get_variable("bias_transform", shape=[d], dtype='float')
# #     W_ = tf.get_variable("weight_", shape=[d, d], dtype='float')
# #     b_ = tf.get_variable("bias_", shape=[d], dtype='float')
#     W_T = tf.Variable(tf.truncated_normal(shp, stddev=0.1), name="weight_transform")
#     b_T = tf.Variable(tf.constant(carry_bias, shape=[d]), name="bias_transform")

#     W_ = tf.Variable(tf.truncated_normal(shp, stddev=0.1), name="weight")
#     b_ = tf.Variable(tf.constant(0.1, shape=[d]), name="bias")
carry_bias=0.
with tf.variable_scope('highway'):
    W_T = tf.Variable(tf.truncated_normal([200,200], stddev=0.1), name="weight_transform")
    b_T = tf.Variable(tf.constant(carry_bias, shape=[200]), name="bias_transform")
    W_ = tf.Variable(tf.truncated_normal([200,200], stddev=0.1), name="weight")
    b_ = tf.Variable(tf.constant(0.1, shape=[200]), name="bias")

    T_x = tf.sigmoid(tf.matmul(xxxx,W_T) + b_T, name="transform_gate")
    H_x = tf.nn.relu(tf.matmul(xxxx,W_) + b_, name="activation")
    C_x = tf.subtract(1.0, T_x, name="carry_gate")
    xxxx = tf.add(tf.multiply(H_x, T_x), tf.multiply(xxxx, C_x), "highway_xx")

    tf.get_variable_scope().reuse_variables()
    T_q = tf.sigmoid(tf.matmul(qqqq,W_T) + b_T, name="transform_gate")
    H_q = tf.nn.relu(tf.matmul(qqqq,W_) + b_, name="activation")
    C_q = tf.subtract(1.0, T_q, name="carry_gate")
    qqqq = tf.add(tf.multiply(H_q, T_q), tf.multiply(qqqq, C_q), "highway_qq")
    xx=tf.reshape(xxxx,[5,400,200])
    qq=tf.reshape(qqqq,[5,30,200])

In [108]:
print(xx.shape)
print(qq.shape)

(5, 400, 200)
(5, 30, 200)


In [109]:
hidden_size=100

In [110]:
from tensorflow.nn.rnn_cell import LSTMCell,DropoutWrapper
from tensorflow.nn import bidirectional_dynamic_rnn

In [111]:
cell=LSTMCell(hidden_size)
cell=DropoutWrapper(cell,output_keep_prob=0.7)

In [112]:
x_len=batch_size
q_len=batch_size

In [113]:
(fw_x,bw_x),_ = bidirectional_dynamic_rnn(cell,cell, xx, dtype ='float32')
(fw_q,bw_q),_ = bidirectional_dynamic_rnn(cell,cell, qq, dtype ='float32')

In [114]:
print(fw_x.shape)
print(bw_x.shape)

(5, 400, 100)
(5, 400, 100)


In [184]:
h=tf.concat([fw_x,bw_x],2)
u=tf.concat([fw_q,bw_q],2)

In [185]:
print(w_s.shape)
print(h.shape)
print(u.shape)
# print(u.shape)

(600, 1)
(5, 400, 200)
(5, 30, 200)


In [186]:
uu=tf.reshape(u,[-1,2*word_emb_size])
hh=tf.reshape(h,[-1,2*word_emb_size])
print(uu.shape)
print(hh.shape)

(150, 200)
(2000, 200)


In [189]:
uk=tf.tile(u,[1,1,400])
uk=tf.reshape(uk,[5,30,400,200])
hk=tf.tile(h,[1,1,30])
hk=tf.reshape(hk,[5,400,30,200])
uk=tf.transpose(uk,perm=[0,2,1,3])
print(uk.shape)
print(hk.shape)
element_mul=tf.multiply(uk,hk)
element_mul.shape
sim_mat=tf.concat([hk,uk,element_mul],3)
print(sim_mat.shape)
sim_mat=tf.reshape(sim_mat,[-1,600])
print(sim_mat.shape)

(5, 400, 30, 200)
(5, 400, 30, 200)
(5, 400, 30, 600)
(60000, 600)


In [153]:
# uuu=tf.tile(uu,[30,1])
# # uuu=tf.reshape(uuu,[2000,30,200])
# print(uuu.shape)
# hhh=tf.tile(hh,[400,1])
# # hhh=tf.reshape(hhh,[150,400,200])
# # hhh=tf.transpose(hhh,perm=[1,0,2])
# print(hhh.shape)
# element_mul=tf.multiply(uuu,hhh)
# element_mul.shape
# sim_mat=tf.concat([h,u,element_mul],2)
# print(sim_mat.shape)
# sim_mat=tf.reshape(sim_mat,[-1,600])
# print(sim_mat.shape)

(60000, 200)
(60000, 200)


In [122]:
w_s=tf.get_variable("weight_sim_mat", shape=[6*word_emb_size,1], dtype='float')

In [190]:
S=tf.matmul(sim_mat,w_s)
S=tf.reshape(S,[batch_size,400,30,1])
S.shape

TensorShape([Dimension(5), Dimension(400), Dimension(30), Dimension(1)])

In [165]:
from tensorflow.nn import softmax

In [219]:
a=softmax(S)
a=tf.tile(a,[1,1,1,200])
a.shape

TensorShape([Dimension(5), Dimension(400), Dimension(30), Dimension(200)])

In [234]:
u_til=tf.multiply(a,uk)
print(u_til.shape)
u_til=tf.reduce_sum(u_til,axis=2)
print(u_til.shape)

(5, 400, 30, 200)
(5, 400, 200)


In [255]:
b=softmax(tf.reduce_max(S,2))
b=tf.tile(b,[1,1,200])
print(b.shape)
h_til=tf.multiply(b,h)
h_til=tf.reduce_sum(h_til,axis=1)
h_til=tf.tile(h_til,[1,400])
h_til=tf.reshape(h_til,[5,200,400])
h_til=tf.transpose(h_til,[0,2,1])
print(h_til)

(5, 400, 200)
Tensor("transpose_15:0", shape=(5, 400, 200), dtype=float32)


In [256]:
print(h.shape)
print(u_til.shape)
print(h_til.shape)

(5, 400, 200)
(5, 400, 200)
(5, 400, 200)


In [258]:
mul_h_u_til=tf.multiply(h,u_til)
print(mul_h_u_til.shape)
mul_h_h_til=tf.multiply(h,h_til)
print(mul_h_h_til.shape)
G=tf.concat([h,u_til,mul_h_u_til,mul_h_h_til],2)
print(G.shape)

(5, 400, 200)
(5, 400, 200)
(5, 400, 800)


In [262]:
cell2=LSTMCell(hidden_size)
cell2=DropoutWrapper(cell2,output_keep_prob=0.7)
(fw_G,bw_G),_ = bidirectional_dynamic_rnn(cell2,cell2, G, dtype ='float32',scope='g0')

In [263]:
M=tf.concat([fw_G,bw_G],2)
print(M.shape)

(5, 400, 200)


In [268]:
G_M=tf.concat([G,M],2)
print(G_M.shape)
w_p1=tf.get_variable("w_p1", shape=[10*word_emb_size], dtype='float')
print(w_p1.shape)

(5, 400, 1000)
(1000,)


In [270]:
p1=softmax(tf.tensordot(G_M,w_p1,[2,0]))
print(p1.shape)

(5, 400)


In [273]:
cell3=LSTMCell(hidden_size)
cell3=DropoutWrapper(cell3,output_keep_prob=0.7)
(fw_M,bw_M),_ = bidirectional_dynamic_rnn(cell3,cell3, M, dtype ='float32',scope='M2_')
M2=tf.concat([fw_M,bw_M],2)
print(M2.shape)

(5, 400, 200)


In [275]:
w_p2=tf.get_variable("w_p2_", shape=[10*word_emb_size], dtype='float')
print(w_p2.shape)

(1000,)


In [276]:
G_M2=tf.concat([G,M2],2)
p2=softmax(tf.tensordot(G_M2,w_p2,[2,0]))
print(p2.shape)

(5, 400)


In [280]:
loss=tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=p1)
loss+=tf.nn.softmax_cross_entropy_with_logits(labels=y2,logits=p2)

In [281]:
loss

<tf.Tensor 'add:0' shape=(5,) dtype=float32>